# Elasticsearch requests/responses example

First of all, you need ES server running on localhost/AWS/somewhere (or ask your friends for one:))

In [1]:
ES_SERVER = raw_input("Enter your URL here: ") 

Enter your URL here: https://search-test-domain-f4jevxgnpbqp22eyx5oyua4ht4.us-east-1.es.amazonaws.com


Let's check if connection to ES server is OK. We'll just post request there and see response

In [2]:
import requests
import json

response = requests.get(ES_SERVER)
json.loads(response.content)

{u'cluster_name': u'202165295345:test-domain',
 u'name': u'Dazzler',
 u'tagline': u'You Know, for Search',
 u'version': {u'build_hash': u'0944b4bae2d0f7a126e92b6133caf1651ae316cc',
  u'build_snapshot': False,
  u'build_timestamp': u'2016-05-20T07:46:04Z',
  u'lucene_version': u'5.5.0',
  u'number': u'2.3.2'}}

Let's define index/type we'll use in this example and update endpoint to ES

In [3]:
index = raw_input("Enter ES index: ")
es_type = raw_input("Enter ES type: ")

es_url = "{}/{}/{}".format(ES_SERVER, index, es_type)

Enter ES index: habrahabr
Enter ES type: article


Now let's try to search for something in cluster

In [4]:
query = raw_input("Enter your query here: ")

url = "{}/_search?q={}".format(es_url, query)

response = requests.get(url)
json.loads(response.content)

Enter your query here: deep learning


{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'cG9zdC8zMDg5OTQ=',
    u'_index': u'habrahabr',
    u'_score': 0.5665711,
    u'_source': {u'article_category': u'\u0420\u0430\u0437\u0440\u0430\u0431\u043e\u0442\u043a\u0430',
     u'article_content': u'\u041a\u0438\u0442\u0430\u0439\u0441\u043a\u0438\u0439 \u043f\u043e\u0438\u0441\u043a\u043e\u0432\u044b\u0439 \u0433\u0438\u0433\u0430\u043d\u0442 Baidu \u043e\u043f\u0443\u0431\u043b\u0438\u043a\u043e\u0432\u0430\u043b \u0434\u0435\u043c\u043e-\u0432\u0435\u0440\u0441\u0438\u044e \u0438\u0441\u0445\u043e\u0434\u043d\u043e\u0433\u043e \u043a\u043e\u0434\u0430 \u0441\u0432\u043e\u0435\u0433\u043e \u0438\u043d\u0441\u0442\u0440\u0443\u043c\u0435\u043d\u0442\u0430 \u0433\u043b\u0443\u0431\u0438\u043d\u043d\u043e\u0433\u043e \u043e\u0431\u0443\u0447\u0435\u043d\u0438\u044f (PArallel Distributed Deep LEarning) \u043d\u0430 . \u0410\u043d\u043e\u043d\u0441 \u0432\u0441\u0435\u0433\u043e \u043a\u043e\u

These requests can be also implemented using **POST** request. In this case we need to specify request body. Let's try to send request to find required word and extract only one field from index. We also will define **SIZE** of response (*default=5*)

In [11]:
field = raw_input("Enter field name that should be extracted: ")
word = raw_input("Enter word to search: ")
size = raw_input("Enter size of response: ")

url = "{}/_search".format(es_url)
data = {
    "query": {
        "query_string": {
            "query": word,
            "analyze_wildcard": True
        }
    },
    "size": int(size),
    "fields": field
}
data = json.dumps(data)
response = requests.get(url, data=data)
json.loads(response.content)

Enter field name that should be extracted: article_url
Enter word to search: python
Enter size of response: 10


{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'Y29tcGFueS9tYWlscnUvYmxvZy8zMDk1NzY=',
    u'_index': u'habrahabr',
    u'_score': 0.5402333,
    u'_type': u'article',
    u'fields': {u'article_url': [u'https://habrahabr.ru/company/mailru/blog/309576/']}},
   {u'_id': u'Y29tcGFueS9Wb3hpbXBsYW50L2Jsb2cvMzA4NzI2',
    u'_index': u'habrahabr',
    u'_score': 0.52290654,
    u'_type': u'article',
    u'fields': {u'article_url': [u'https://habrahabr.ru/company/Voximplant/blog/308726/']}},
   {u'_id': u'cG9zdC8zMDc2OTI=',
    u'_index': u'habrahabr',
    u'_score': 0.4720181,
    u'_type': u'article',
    u'fields': {u'article_url': [u'https://habrahabr.ru/post/307692/']}},
   {u'_id': u'cG9zdC8zMDk2NjA=',
    u'_index': u'habrahabr',
    u'_score': 0.38200265,
    u'_type': u'article',
    u'fields': {u'article_url': [u'https://habrahabr.ru/post/309660/']}},
   {u'_id': u'Y29tcGFueS9waXRlci9ibG9nLzMwNzgzOA==',
    u'_index': u'habrahabr',
    u'_s

To extract exact required result we'll dig into dict under "hits" key. Then values in "hits" again... Given list will be response to our query

In [13]:
hits = json.loads(response.content)["hits"]["hits"]
for hit in hits:
    print "{} - score: {}".format(hit["fields"].values()[0][0], hit["_score"])

https://habrahabr.ru/company/mailru/blog/309576/ - score: 0.5402333
https://habrahabr.ru/company/Voximplant/blog/308726/ - score: 0.52290654
https://habrahabr.ru/post/307692/ - score: 0.4720181
https://habrahabr.ru/post/309660/ - score: 0.38200265
https://habrahabr.ru/company/piter/blog/307838/ - score: 0.37181485
https://habrahabr.ru/company/intel/blog/305126/ - score: 0.36941257
https://habrahabr.ru/post/308162/ - score: 0.36311066
https://habrahabr.ru/company/Voximplant/blog/307848/ - score: 0.2833762
https://habrahabr.ru/post/309274/ - score: 0.27924967
https://habrahabr.ru/post/309712/ - score: 0.26386613
